## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# Microsoft Movie Studio Projest

## 1.Business Understanding

Microsoft has started a new movie studio and to know which movies do the best they have enlisted services of a data scientist.To come up with results that will enable the movie studio to make this decision, I as the data scientist will look at available resources such as the imdb to come up with observations and recommendations that are relevant to MIcrosoft

# 2.Data Understanding

For this operation we will be looking mainly at two sources of data namely the IMDb and The Box Office Mojo data which cointain alot of info on movies from cast all the way to budget

In [117]:
# below we will import all the libraries which will enable us to handle our data
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile


Since the data in IMDb is zipped it has to be unzipped first

In [118]:
# Unzip using zipfile
path = "zippedData/im.db.zip"
with zipfile.ZipFile(path, 'r') as zip_loc:
    zip_loc.extractall('zara')

Now we can open the the database using sqlite3

In [119]:
#Make a connection to the database
conn = sqlite3.connect("zara/im.db")

#USing pandas get the table names in the database
q ="""SELECT name 
      FROM sqlite_master 
      WHERE type = 'table'""";

pd.read_sql(q, conn)


,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


From the data base the most relevant tables to use are the movie_basics and movie_ratings.Lets take a look at the tables and familiarise ourselves with their content    

In [120]:
#Load the content of the table(columns and rows) from movie_basics
#Load the first 5 rows
pd.read_sql("""SELECT * FROM movie_basics;""", conn).head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [121]:
#Load the last 5 rows from movie_ratings
pd.read_sql("""SELECT * FROM movie_ratings;""", conn).tail()

,movie_id,averagerating,numvotes
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5
73855,tt9894098,6.3,128


From the looks of it have one-to-one relationship on the movie_id column which seems to be the primary key in both



Now we look at the structure of the two tables

In [122]:
#use .info()
pd.read_sql("""SELECT * FROM movie_basics;""", conn).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [123]:
pd.read_sql("""SELECT * FROM movie_ratings;""", conn).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


We can also take a look at the descriptive ststistics of the tables

In [124]:
#Getting measures of central tendancies and dispersion
pd.read_sql("""SELECT * FROM movie_basics;""", conn).describe()

,start_year,runtime_minutes
count,146144.000000,114405.000000
mean,2014.621798,86.187247
std,2.733583,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2115.000000,51420.000000


In [125]:
pd.read_sql("""SELECT * FROM movie_ratings;""", conn).describe()

,averagerating,numvotes
count,73856.000000,7.385600e+04
mean,6.332729,3.523662e+03
std,1.474978,3.029402e+04
min,1.000000,5.000000e+00
25%,5.500000,1.400000e+01
50%,6.500000,4.900000e+01
75%,7.400000,2.820000e+02
max,10.000000,1.841066e+06


In [161]:
#loking at the shape of the tables
basic = pd.read_sql("""SELECT * FROM movie_basics;""", conn)
rate = pd.read_sql("""SELECT * FROM movie_ratings;""", conn)

print('The movie_basics table has ' + str(basic.shape[0]) + ' rows and ' + str(basic.shape[1]) + 'columns')
print('The movie_ratings table has ' + str(rate.shape[0]) + ' rows and ' + str(rate.shape[1]) + 'columns')

The movie_basics table has 146144 rows and 6columns
The movie_ratings table has 73856 rows and 3columns


Another source of data is The Box Office Mojo csv file.Repeat the same process above on the file

In [127]:
# Load the csv into a pandas dataframe 
bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
bom

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [128]:
#Looking at the structure of the data set 
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [129]:
#Getting thr drescriptive statistics of the dataset
bom.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [130]:
print('The dataset has ' + str(bom.shape[0]) + ' rows and ' + str(bom.shape[1]) + ' columns')

The dataset has 3387 rows and 5 columns


Now that we know what our datasets contain and their features we can begin on cleaning it up for analysis

# Data Cleaning

Checking for Validity, Accuracy, Completeness, Consistency and Uniformity of the Data.
Data cleaning is necessary in order to ensure the accuracy of the final result of data analysis.To ensure this the consistency,quality and validity of the data has to be checked


The first step is checking if the datatypes of all the columns makes sense to our data

In [131]:
#For all datasets check columns Datatypes
basic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [132]:
rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [133]:
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


* As it can be seen above the foreign_gross column datatype is an object while it would be more relevant for it to be a float.Below the      datatypes will be changed

In [134]:
#strip the objects of an punctuation or white space and change
#the column type from object to float

bom['foreign_gross'] = bom['foreign_gross'].str.replace(',','').astype(float)

In [138]:
#Check to see if changes applied
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   float64
 4   year            3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


Duplicates Should as be checked for

In [136]:
#Identify datasets with with duplicates
print('The movies_basics has ' + str(basic.duplicated().sum()) + ' duplicates')
print('The movies_ratings has ' + str(rate.duplicated().sum()) + ' duplicates')
print('The Box Office Mojo dataset has ' + str(basic.duplicated().sum()) + ' duplicates')

The movies_basics has 0 duplicates
The movies_ratings has 0 duplicates
The Box Office Mojo dataset has 0 duplicates


Null values in your datasets can also have a major impact on the accuracy of data analysis and can make the recommendations misleading


Next check for null values in the datasets

In [137]:
#If any check for the number of null values in each column
basic.isnull().sum()

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

In [163]:
rate.isnull().sum()

movie_id         0
averagerating    0
numvotes         0
dtype: int64

In [160]:
bom.isnull().sum()

title                0
studio               0
domestic_gross      26
foreign_gross     1349
year                 0
dtype: int64

* As it can be see above both the movie_basics table and the B.O.M dataset have missing values which have to be delt with
* To deal with the columns with a relatively small number of values, the rows with with those missing values in those respective columns can be droped.

In [166]:
#Drop the rows with missing values in the genre and original_title columns in the movie_ratings table 
basic.dropna(subset= ['genres'], inplace=True)
basic.dropna(subset= ['original_title'], inplace=True)

In [152]:
#Drop the rows with missing values in studio column in the B.O.M dataframe
bom.dropna(subset= ['studio'], inplace=True)

* For the remaining columns the process will be replacing the null values with the median of the values in the column
* The reason for this is because the median has less effect on the distribution of the data set

In [170]:
#Replace the null values in the domestic_gross and foreign_gross column with the column's median
bom['domestic_gross'].fillna(bom['domestic_gross'].median(), inplace=True)
bom['foreign_gross'].fillna(bom['foreign_gross'].median(), inplace=True)

In [174]:
#Replace the null values in the runtime_minutes column with the column's median
basic['runtime_minutes'].fillna(basic['runtime_minutes'].median(), inplace=True)